## Zadania: LangChain: Chat z pamięcią

> **Wskazówka:** miejsca do uzupełnienia oznaczono jako `# TODO` lub `...`.


## Zadanie 1 - `RunnableWithMessageHistory`: opakuj łańcuch o pamięć

In [3]:
# Cel: Zbuduj prosty łańcuch z pamięcią konwersacji.
# Odniesienie: 2_4_LangChain_chat_with_memory.ipynb

import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from dotenv import load_dotenv

load_dotenv()

# Upewnij się, że masz ustawione OPENAI_API_KEY w środowisku.
# os.environ["OPENAI_API_KEY"] = "sk-..."  # NIE wstawiaj klucza do repo

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # TODO: możesz zmienić model

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś asystentem. Pamiętaj kontekst rozmowy."),
    MessagesPlaceholder("history"),
    ("user", "{input}"),
])

chain = prompt | llm  # TODO: prosta kompozycja

store = {}
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
)

# (Nie wywołujemy realnie modelu w testach); Sprawdź, czy obiekt istnieje.
assert chain_with_memory is not None, "Nie udało się zbudować łańcucha z pamięcią."

## Zadanie 2 - Pamięć rozmowy: dodaj bufor rozmowy do łańcucha
*Uwaga:* interfejs pamięci w LangChain ewoluuje. Poniższy szkic pokazuje ideę z buforem rozmów.

In [4]:
# Cel: Zademonstruj integrację bufora historii z łańcuchem (wersja poglądowa).
# Jeśli Twoja wersja LangChain oferuje nowsze podejście, użyj go.

try:
    from langchain.memory import ConversationBufferMemory  # API może się różnić w wersjach
    memory = ConversationBufferMemory(return_messages=True)
    memory.save_context({"input": "Hej!"}, {"output": "Cześć, w czym pomóc?"})
    hist = memory.load_memory_variables({})
    print("Historia:", hist)
except Exception as e:
    print("Pominięto demonstrację ConversationBufferMemory (inne API?):", e)

Historia: {'history': [HumanMessage(content='Hej!', additional_kwargs={}, response_metadata={}), AIMessage(content='Cześć, w czym pomóc?', additional_kwargs={}, response_metadata={})]}


/tmp/ipykernel_21449/1815453156.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
